In [ ]:
cd '/content/drive/MyDrive/GAN-based-Face-Unmasking/detector'

/content/drive/.shortcut-targets-by-id/1zskoOzbVHJNUjVdJrfPFNsqRuPe-cGUB/GAN-based-Face-Unmasking/detector


In [ ]:
import pickle
import torch
import torch.nn as nn

import detect_model
from metrics import pixelacc
from torch.utils.tensorboard import SummaryWriter

In [ ]:
#tensor dataset 불러오기
with open("detect_train_data.pickle","rb") as fr:
    train = pickle.load(fr)
with open("detect_label_data.pickle","rb") as fr:
    label = pickle.load(fr)

with open("detect_test_data.pickle","rb") as fr:
    test = pickle.load(fr)
with open("detect_tlabel_data.pickle","rb") as fr:
    tlabel = pickle.load(fr)

In [ ]:
summary = SummaryWriter()
########batch tensor 설정########
#train, label
#test, tlabel
#순서
#1. img를 하나씩 넣는 [x] 준비
#2. [x]들을 가지고 있는 img개수가 batch_size가 되면 cat 후 train_dataset에 넣음

train_size=len(train)
train_dataset=[]
#batch,concat용 리스트
train_batch=[train[0]]
label_batch=[label[0]]
batch_size = 2
if batch_size==1:
    train_temp = torch.cat(train_batch)
    label_temp = torch.cat(label_batch)
    train_dataset.append([train_temp,label_temp])
    train_batch, label_batch = [],[]
for i in range(1,train_size):
    if (i+1)%batch_size == 0:
        train_batch.append(train[i])
        label_batch.append(label[i])
        
        train_temp = torch.cat(train_batch)
        label_temp = torch.cat(label_batch)
        train_dataset.append([train_temp,label_temp])
        train_batch, label_batch = [],[]
    else:
        train_batch.append(train[i])
        label_batch.append(label[i])
    

test_size=len(test)
test_dataset=[]
#batch,concat용 리스트
test_batch=[test[0]]
tlabel_batch=[tlabel[0]]
test_batch_size = 1
if test_batch_size==1:
    test_temp = torch.cat(test_batch)
    tlabel_temp = torch.cat(tlabel_batch)
    test_dataset.append([test_temp,tlabel_temp])
    test_batch, tlabel_batch = [],[]
for i in range(1,test_size):
    if (i+1)%test_batch_size == 0:
        test_batch.append(test[i])
        tlabel_batch.append(tlabel[i])
        
        test_temp = torch.cat(test_batch)
        tlabel_temp = torch.cat(tlabel_batch)
        test_dataset.append([test_temp,tlabel_temp])
        test_batch, tlabel_batch = [],[]
    else:
        test_batch.append(test[i])
        tlabel_batch.append(tlabel[i])


#학습 파라미터 설정
#img_size = 128
in_dim = 3
out_dim = 1
num_filters = 64
num_epoch = 300
lr = 0.001

patience=30
flag = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = detect_model.Detector(in_dim=in_dim,out_dim=out_dim,num_filter=num_filters).to(device)

loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

import time
val_loss = 1
early_stopping = 0
for i in range(num_epoch+1):
    start_time = time.time()
    #######train###########
    for j in range(len(train_dataset)):
        X = train_dataset[j][0]
        y = train_dataset[j][1]
        
        optimizer.zero_grad()
        output = model.forward(X)
        
        loss = loss_func(output,y)
        loss.backward()
        
        optimizer.step()
    
    #######test###########
    metrics = [pixelacc.PixelAccuracy(1)]
    test_loss, correct = 0, 0
    with torch.no_grad():
        for tX, ty in test_dataset:
            pred = model(tX)
            test_loss += loss_func(pred, ty).item()
            for metric in metrics:
                metric.update(pred.cpu(), ty.cpu())
                

    test_loss /= len(test_dataset)
    end_time = time.time()
    running_time = end_time - start_time

    summary.add_scalar('loss', loss.item(), i+1)
    summary.add_scalar('test_loss', test_loss, i+1)
    
    print(f"epoch:{(i+1):4d}  ||  train Loss:{loss.item():.6f}  ||  test Loss:{test_loss:.6f}  ||  time:{running_time*5:.3f}")
    for metric in metrics:
        print(metric)
    if (i+1) % 3 ==0:
        if test_loss < val_loss:
            print(f'loss advanced : {val_loss:.6f} --> {test_loss:.6f}___model saved___!!!!!!')
            val_loss = test_loss
            torch.save(model.state_dict(), f'/content/drive/MyDrive/GAN-based-Face-Unmasking/detector/weights/detect_b2_2_{i+1}epochs.pth')
        else: 
            early_stopping+=1
            if early_stopping >= 40:
                break

summary.close()
print("Done!")


------Initiating Detect, U-Net------



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch:   1  ||  train Loss:0.018041  ||  test Loss:0.036815  ||  time:1115.728
Pixel Accuracy: 74.83999999999999
epoch:   2  ||  train Loss:0.015138  ||  test Loss:0.025483  ||  time:1115.068
Pixel Accuracy: 74.51
epoch:   3  ||  train Loss:0.014048  ||  test Loss:0.024013  ||  time:1114.891
Pixel Accuracy: 74.57000000000001
loss advanced : 1.000000 --> 0.024013___model saved___!!!!!!
epoch:   4  ||  train Loss:0.014027  ||  test Loss:0.022871  ||  time:1114.580
Pixel Accuracy: 74.53
epoch:   5  ||  train Loss:0.013365  ||  test Loss:0.022550  ||  time:1114.355
Pixel Accuracy: 74.61
epoch:   6  ||  train Loss:0.013393  ||  test Loss:0.021809  ||  time:1114.149
Pixel Accuracy: 74.59
loss advanced : 0.024013 --> 0.021809___model saved___!!!!!!
epoch:   7  ||  train Loss:0.013338  ||  test Loss:0.021504  ||  time:1114.058
Pixel Accuracy: 74.57000000000001
epoch:   8  ||  train Loss:0.013083  ||  test Loss:0.021689  ||  time:1114.375
Pixel Accuracy: 74.57000000000001
epoch:   9  ||  train 